# <center> HADOOP STACK
<img src="HadoopStack.png">

# <center> **QUE ES HADOOP** </center>

# Es un framework para **Almacenamiento y Procesamiento de lotes con altos volumenes de datos** de manera distribuida y ejecucion de tareas en paralelo

# <center> **COMPONENTES HADOOP** </center>
<div style="text-align:center">
    <img src="HadoopComponentes.png">
</div>

# <center> **HADOOP COMMON** </center>
<div style="text-align:center">
    <img src="commons.png">
</div>

## - Contiene los archivos y scripts Java Archive (JAR) necesarios para iniciar Hadoop.
## - Colección de utilidades y bibliotecas comunes que son necesarias para los otros módulos de Hadoop.
## - Proporciona código fuente y documentación. 
## - Cuenta con una sección de contribuciones que contienen multiples proyectos de la comunidad de Hadoop.

# <center> **HADOOP YARN** </center>

## **YARN** funciona a través de un **Resource Manager** que es uno por nodo y un **Node Manager** que se ejecuta en todos los nodos.

* **Resource Manager** este **corre en el master node**. Administra los recursos utilizados en el clúster, **provee un endpoint a los clientes para hacer la solicitud de envio de Jobs** Hace seguimiento del heart beats del Node Manager.
    * **Scheduler**  este **decide cómo se asignan los recursos a varias tareas.** Programa los recursos a las **aplicaciones en ejecución en función de los requisitos, la capacidad y la cola**. **No garantiza reiniciar las tareas fallidas**.
    * **Application Manager** administra la ejecución de **Application Master en un clúster** y, en caso de falla del contenedor de Application Master, **ayuda a reiniciarlo**. Asimismo, tiene la responsabilidad de aceptar los Jobs enviados.

   
* **Node Manager** este **corre en el slave node**, Monitorea los **Containers** y Crea **Containers** basado en los requerimientos de la Task, es Responsable de la **ejecución de las Tasks en cada DataNode**. envia **heart beats al Resource Manager**

* **Containers** Conjunto de recursos como **RAM, CPU, GPU, Disco, Red** de un Nodo o **Host del Cluster**. Son programados por **Resource Manager** y monitoreados por **Node Manager**.

* **Application Master** Es **creado por un Job** y controla la monitorización y la **ejecución de las tareas usando el Container**, es el que **Negocia los recursos para el Job con el Resource Manager**. Gestiona el ciclo de vida de los **Jobs que se ejecutan en el clúster hasta que se completen**.

* **History Server** es el encargado de **Almacenar el historial de todos los Jobs**.

<div style="text-align:center">
    <img src="yarn.png">
</div>

# <center> HDFS **HADOOP DISTRIBUTE FILE SYSTEM** Y HCFS **HADOOP COMPATIBLE FILE SYSTEM** </center>
# Ss un sistema de archivos distribuido que proporciona espacio de almacenamiento redundante para archivos de gran tamaño especialmente en el rango de terabytes a petabytes.
<div style="text-align:center">
    <img src="hdfs.png">
</div>

# <center> ARCHIVOS</center>

- Los Archivos que se guardan en el **HDFS son inmutables** y se pueden leer tantas veces como se quiera, por la inmutabilidad no se pueden modificar los archivos.
- Los archivos que se van a almacenar en el HDFS **se recomienda que sean de 128 Megas** o mas.

- Muchos **Archivos pequeños hacen ineficientes el procesamiento** y el almacenamiento, hay varias razones por las cuales se generan archivos de bajo peso como:
    * Los archivos pueden ser la pieza de un archivo lógico más grande. 
    * Algunos archivos no se pueden combinar en un archivo más grande y son esencialmente pequeños. aunque se comprima un set de imagenes en un archivo que ocupe las 128 megas recomendadas cuando se procesen estas imagenes, se hara de manera separada imagen por imagen, ya que donde cada imagen es un archivo distinto, haciendo ineficiente el proceso.
    

- **Efectos de los archivos pequeños**: 
    - HDFS no puede manejar de manera eficiente el almacenamiento de muchos archivos pequeños que son extremadamente pequeños que el tamaño del bloque. 
    - La lectura de archivos pequeños implica muchas búsquedas y muchos saltos entre nodos de datos, lo que a su vez es un procesamiento de datos ineficiente.
    - En la memoria de namenode, cada archivo, directorio y bloque en HDFS se representa como un objeto. Cada uno de estos objetos tiene un tamaño de 150 bytes.
    - Si consideramos 10 millones de archivos pequeños, cada uno de estos archivos usará un bloque separado. Eso provocará un uso de 3 gigas de memoria.
    - Con las limitaciones de hardware que tiene, escalar más allá de este nivel es un problema. 
    - Con muchos archivos, la memoria requerida para almacenar los metadatos es alta y no se puede escalar más allá de un límite.
    - En MapReduce, la tarea Map procesa un bloque de datos a la vez. 
    - Muchos archivos pequeños significan muchos bloques, lo que significa muchas tareas y mucha contabilidad por parte de Application Master.
    - Esto ralentizará el rendimiento general del clúster en comparación con el procesamiento de archivos grandes.

    **Alternativas**:

    **1\. Archivos Hadoop, HAR**:
     - Hadoop Archives o HAR es una instalación de archivo que empaqueta archivos en bloques HDFS de manera eficiente. No relacione los archivos .har con archivos comprimidos.
     - HAR se crea a partir de una colección de archivos y la herramienta de archivo (un comando simple) ejecutará un trabajo de MapReduce para procesar los archivos de entrada en paralelo y crear un archivo de archivos.
     - los pequeños archivos dentro del archivo .har serán procesados individualmente por mapeadores separados, lo cual es ineficiente.

    **2\. SequenceFile System**:
     * Use el nombre de cada archivo como clave y el contenido del cada archivo como valor.
     * Puede ser procesados en forma de Streaming (directamente o usando MapReduce) operando en SequenceFile
     * Se pueden dividir, por lo que MapReduce puede dividirlos en fragmentos y operar en cada fragmento de forma independiente.
     * Admiten compresión, a diferencia de los HAR. La compresión de bloques es la mejor opción en la mayoría de los casos, ya que comprime bloques de varios registros (en lugar de por registro).

    **3\. HBase si no es un problema la latencia**:
     - Si está produciendo muchos archivos pequeños, entonces, dependiendo del patrón de acceso, un tipo diferente de almacenamiento podría ser más apropiado.
     - Almacena datos en MapFiles (SequenceFiles indexados) y es una buena opción si necesita realizar análisis Streaming de estilo MapReduce con la búsqueda aleatoria ocasional.
     
* **Secundary Namenode** Asume la responsabilidad de fusionar los registros de edición con fsimage del namenode. Es Como un LogShipping cronometrado.
* **StandBy Namenode** Releva al namenode cuando este falla. Es como Streaming Replication con failover integrado.

# <center> TIPOS DE ARCHIVOS</center>

- Sequential files
- Text
- Avro : sistema de serialización de datos, en un formato binario compacto independiente del Lenguaje. Usa esquemas basados en JSON.  utiliza llamadas RPC para enviar datos, durante el intercambio de datos, se envia el esquema. 
- ORC
- Parquet
- Arrow
- Protocol Buffers
- Thrift

# <center> ALGORITMOS DE COMPRESION</center>
- Snappy
- Zlib
- Gzip
- LZO
- Bzip2

<div style="text-align:center">
    <img src="cluster.png">
</div>

<div style="text-align:center">
    <img src="HadoopVentajas.png">
</div>

# <center> **MAP-REDUCE**</center>
<div style="text-align:center">
    <img src="mapreduce.png">
</div>
- Secondary Sort : Ordena los valores de la fase de Mapping antes de entregarsela al reductor de manera ascendente o descendente, en los casos que sean necesarios por ejemplo para una serie de tiempo.
- Join: Son usados para unir dos o más conjuntos de datos en función de alguna columna.
    - Map Side Join:
        - la operación de Join se realiza en la fase Map.
        - Se usa cuando un conjunto de datos es grande y el otro conjunto de datos es pequeño.
        - Ordenado por la misma clave.
        - Igual número de particiones.
        - Todos los registros de la misma clave deben estar en la misma partición.
    - Reduce Side Join:
        - la operación de Join se realiza en la fase Reduce.
        - Se usa cuando ambos conjunto de datos son grandes.
        - Mucho más flexible de implementar.
        - Tiene que haber WritableComparable personalizado con la función necesaria anulada.
        - Necesitamos un particionador personalizado.
        - Se requiere un comparador de grupo personalizado.
    
**Si tiene experiencia en SQL, no necesita preocuparse por escribir el código MapReduce Java para realizar una operación de unión. Puede utilizar Hive como alternativa.**
    
- Chained Mappers: permite usar múltiples clases de Mapper dentro de una sola tarea de Map.
- Chained Reducer permite encadenar varias clases de Mapper después de un Reducer dentro de la tarea Reducer.
- Bloom Filter
- In memory Mapper/Reducer side join
  

# <center>FASES DEL PARADIGMA **MAP-REDUCE**
<img src="MapReduce-fases.png">

# <center> DEFINE LOS DATOS CON QUE SE VA A GENERAR LA INFORMACION DE ENTRADA

In [ ]:
import random
valores = ['CERO','UNO','DOS','TRES','CUATRO','CINCO','SEIS','SIETE','OCHO','NUEVE','DIEZ','ZERO','ONE','TWO','THREE','FOUR','FIVE','SIX','SEVEN','EIGHT','NINE','TEN']
valores = ['CERO','UNO','DOS','TRES','CUATRO','CINCO','SEIS','SIETE','OCHO','NUEVE','DIEZ']

caracter_separador = ' @'
valores

# <center> GENERA LOS DATOS A PROCESAR **INPUT**
<img src="MapReduce-fases.png">

In [ ]:
# retorna una lista con valores
def generar_valores(limite_inferior:int = 2, limite_superior:int = 10,caracter_separador:str = caracter_separador) -> list:
    return [random.choice(valores) for valor in range(random.randint(limite_inferior, limite_superior))]

# retorna una cadena de valores con la informacion a particionar
def generar_archivo_de_datos(lista_valores_entrada:list, cantidad_de_muestras:int = 100)-> str:

  # recupera cada lista generada la convierte a cadena y le adiciona un caracter para usarlo como separador
  texto = [' '.join(generar_valores()) + caracter_separador for limite_listas in range(cantidad_de_muestras)]

  # une el contenido de las listas en una sola cadena
  return ''.join(texto)

archivo_de_datos = generar_archivo_de_datos(generar_valores())
archivo_de_datos

# <center> DIVIDE LOS DATOS A PROCESAR BASADO EN EL CARACTER SEPARADOR **@** **SPLITTING**
<img src="MapReduce-fases.png">

In [ ]:
# divide el archivo en listas
def splitting(arhivo_de_datos) -> list:
  # se pone [:-1] para que no genere una cadena vacia al final
  return arhivo_de_datos.split(caracter_separador)[:-1]

splitting_text = splitting(archivo_de_datos)
splitting_text

# <center> CREA LA INFORMACION DE LOS REGISTROS DEL **RECORDREADER** PARA QUE SEAN USADOS POR EL **MAPPER**
<img src="MapReduce-fases.png">

In [ ]:
def record_reader(input_file):
    record_file = []
    inicio = 0
    for record in input_file:
        
        record_file.append((inicio, record))
        inicio += len(record)
    
    return record_file
        
record_reader_file = record_reader(splitting_text)
record_reader_file

# <center> EJECUTA EL CODIGO DE MAPEO DE LOS DATOS **MAPPING**
<img src="MapReduce-fases.png">

In [ ]:
def lista_a_lista_de_diccionarios(lista):
    return [{str(llave):1} for llave in lista]

# divide cada palabra 1 a 1 en cada splitting text
def mapping(record_reader_file):
    mapeo = []
    
    for record_reader in record_reader_file:
        lista_cadena = record_reader[1].split()
        mapeo.append(lista_a_lista_de_diccionarios(lista_cadena))
        
    return mapeo

lista_mapping = mapping(record_reader_file)
print(lista_mapping)

# <center> COMBINA LOS DATOS DE LAS CLAVES DEL MISMO BLOQUE DE MAPEO **COMBINER**
<img src="MapReduce-fases.png">

In [ ]:
def unir_claves(lista):
    return {list(clave.keys())[0]:lista.count(clave) for clave in lista}

# junta las claves (las une) de cada particion del mapper
def combiner(lista_mapping):
  
  return [unir_claves(lista) for lista in lista_mapping]

combiner_values = combiner(lista_mapping)
print(combiner_values)

# <center> CREA PARTICIONES USANDO EN ESTE CASO COMO BASE DE PARTICION LOS NOMBRES DE LAS CLAVES **PARTITIONER**
<img src="MapReduce-fases.png">

In [ ]:
# divide cada diccionario de un combiner y genera una lista de diccionarios separados
def generar_diccionarios(lista:list):
    return [{key:value} for key,value in lista.items()]

# genera particiones por cada clave basado en el nombre de la clave
def generar_lista_de_diccionarios(combiner_values):
    particion = list()
    
    for particion_valores in combiner_values:
        particion.extend(generar_diccionarios(particion_valores))
    
    return particion
  
# particionado para este ejemplo por clave del diccionarario
def particionar(combiner_values):
    valores_lista_de_diccionarios = generar_lista_de_diccionarios(combiner_values)
    particion = dict()
    for diccionario in valores_lista_de_diccionarios:
        llave = list(diccionario.keys())[0]
        
        if llave in particion:
            valor = particion.get(llave)
            valor.append(diccionario)
            particion[llave] = valor
        else:
            particion[llave] = [diccionario]
            
    return particion

claves_particionadas = particionar(combiner_values)
print(claves_particionadas)

# <center> AGRUPA LOS DATOS Y LOS ORDENA CREANDO UNA LISTA CON CADA VALOR PARA CADA CLAVE **SHUFFLE AND SORT**
<img src="MapReduce-fases.png">

In [ ]:
def shuffle_and_short(claves_particionadas):
    
    particiones_agrupadas = dict()
    
    for llave in claves_particionadas:
        particiones_agrupadas[llave] = [list(valor.values())[0] for valor in claves_particionadas.get(llave)]
        
    return particiones_agrupadas
  

shuffle_and_short_ordenados = shuffle_and_short(claves_particionadas)

print(shuffle_and_short_ordenados)

# <center> EJECUTA EL ALGORITMO DE REDUCCION PARA CADA CLAVE, EN ESTE CASO LOS SUMA **REDUCER**
<img src="MapReduce-fases.png">

In [ ]:
def reducer(shuffle_and_short_ordenados):
    valores_reducidos = dict()
    
    for llave in shuffle_and_short_ordenados:
        
        valores_reducidos[llave] = sum(shuffle_and_short_ordenados.get(llave))
        
    return valores_reducidos

datos_reducer = reducer(shuffle_and_short_ordenados)
datos_reducer

# <center> GENERA EL ARCHIVO DE SALIDA CON LOS DATOS FINALES **OUTPUT**
<img src="MapReduce-fases.png">

In [ ]:
import pandas as pd

def output_dataframe(datos_reducer):

  # crea los valores de las keys en tipo lista para que se puedan adicionar al dataframe sin problemas
  datos_pandas = {key:[value] for key, value in datos_reducer.items()}
  
  return pd.DataFrame(datos_pandas)

output_dataframe(datos_reducer)